In [1]:
import os
import torch
from torch.utils.data import DataLoader
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import Dataset
from torchvision import transforms
import resnet
import torch.nn as nn
import torch.nn.functional as F
from scipy import stats
import gc
import time

/usr/local/home/sgchr/anaconda3/envs/pyt/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/local/home/sgchr/anaconda3/envs/pyt/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [10]:
import os
os.getcwd()
import pickle

In [12]:
with open('/usr/local/home/sgchr/Documents/active-nn/defaultsavefilename.p','rb' ) as f:
    imp_wt_idxs = pickle.load(f)
print(imp_wt_idxs)

{'FISH': [], 'BAIT': [0.1]}


In [2]:
data_tr = datasets.CIFAR10('data'+ '/CIFAR10', train=True, download=True)
data_te = datasets.CIFAR10('data' + '/CIFAR10', train=False, download=True)
X_tr = data_tr.data
Y_tr = torch.from_numpy(np.array(data_tr.targets))
X_te = data_te.data
Y_te = torch.from_numpy(np.array(data_te.targets))

Files already downloaded and verified
Files already downloaded and verified


In [3]:
num_imp_per_layer = 7000
xt = np.zeros((len(Y_tr), len(np.unique(Y_tr)), num_imp_per_layer))

In [4]:
fisher = torch.zeros(xt.shape[-1], xt.shape[-1],dtype=torch.double).cuda()
iterates = torch.zeros(xt.shape[-1], xt.shape[-1], dtype=torch.double)

In [7]:
n_pool = len(Y_tr)
idxs_lb = np.zeros(n_pool, dtype=bool)
idxs_unlabeled = np.arange(n_pool)[~idxs_lb]

In [8]:
chosen = select(xt[idxs_unlabeled], 1000, fisher, iterates, lamb = 1, backwardSteps = 0, nLabeled=np.sum(idxs_lb))

KeyboardInterrupt: 

In [ ]:
class DataHandler3(Dataset):
    def __init__(self, X, Y, transform= None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.X[index], self.Y[index]
        if self.transform is not None:
            x = Image.fromarray(x)
            x = self.transform(x)
        return x, y, index
    def __len__(self):
        return len(self.Y)

In [ ]:
def get_handler(name):
    if name == 'CIFAR10':
        return DataHandler3

In [ ]:
handler = get_handler('CIFAR10')

In [ ]:
print(type(get_handler))

In [ ]:
args={'n_epoch': 3, 'transform': transforms.Compose([ 
                     transforms.RandomCrop(32, padding=4),
                     transforms.RandomHorizontalFlip(),
                     transforms.ToTensor(),
                     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
                 ]),
                 'loader_tr_args':{'batch_size': 128, 'num_workers': 1},
                 'loader_te_args':{'batch_size': 1000, 'num_workers': 1}, # change back to 1000
                 'optimizer_args':{'lr': 0.05, 'momentum': 0.3},
                 'transformTest': transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])}

In [ ]:
def predict_prob(X, Y, model, exp=True):
    # if type(model) == list: model = clf

    # loader_te = DataLoader(handler(X, Y, transform=args['transformTest']), shuffle=False, **args['loader_te_args'])
    # print(type(loader_te))
    loader_te = DataLoader(X_tr, batch_size = 100, shuffle = False)
    model = model.eval()
    probs = torch.zeros([len(Y), len(np.unique(Y))])
    with torch.no_grad():
        for idx, x in enumerate(loader_te):
            x = Variable(x.cuda())
            out, e1 = model(x)
            if exp: out = F.softmax(out, dim=1)
            probs[idx] = out.cpu().data
            # probs = out.cpu().data
    
    return probs

In [ ]:
innerInv = torch.tensor(())

In [ ]:
X_tr[:100].shape[0]

In [ ]:
loader_te = DataLoader(X_tr, batch_size = 100, shuffle = False)
for idx,x in enumerate(loader_te):
    print(x.shape)

In [ ]:
model = resnet.ResNet18()

In [ ]:
phat = predict_prob(X_tr[idxs_unlabeled], Y_tr[idxs_unlabeled],model)

In [ ]:
print('all probs: ' + 
                str(str(torch.mean(torch.max(phat, 1)[0]).item())) + ' ' + 
                str(str(torch.mean(torch.min(phat, 1)[0]).item())) + ' ' + 
                str(str(torch.mean(torch.std(phat,1)).item())), flush=True)

In [1]:
import pickle
savefile = open("defaultsavefilename.p", "br")
save = pickle.load(savefile)
acc_dict = save
print(acc_dict)

{'FISH': [0.0999], 'BAIT': [0.1002, 0.1005]}
